In [103]:
from dotenv import load_dotenv
import os
import requests
load_dotenv('../.env')

True

In [104]:
BEARER_TOKEN = os.getenv('GITHUB_TOKEN')
if BEARER_TOKEN :
    print('GITHUB TOKEN found in envirionment')
else:
   print('GITHUB TOKEN not found')

GITHUB TOKEN found in envirionment


In [105]:
USER = 'Kaggle' # will take input from the user instead hardcoded
BASE_URL = 'https://api.github.com/'


response = requests.get(
    f'{BASE_URL}/users/{USER}/repos',
    headers={
        'Authorization': f'Bearer {BEARER_TOKEN}'
    }
)
repositories = response.json()

In [115]:
class Repository:
    name = ''
    stargazers_count = 0
    forks_count = 0
    
    # further api metrics

    commits_count = 0
    contributors_count = 0
    branches_count = 0
    tags_count = 0
    releases_count = 0
    closed_issues_count = 0
    enviroments_count = 0

    def __init__(self, name, stargazers_count, forks_count):
        self.name = name
        self.stargazers_count = stargazers_count
        self.forks_count = forks_count

        self.commits_count = self.__get_metrics_count('commits')
        self.contributors_count = self.__get_metrics_count('contributors')
        self.branches_count = self.__get_metrics_count('branches')
        self.tags_count = self.__get_metrics_count('tags')
        self.issues_count = self.__get_metrics_count('issues')
        self.releases_count = self.__get_metrics_count('releases')    
        self.enviroments_count = self.__get_metrics_count('environments')    
  
    def __get_metrics_count(self, url_extension):
        url = f'{BASE_URL}/repos/{USER}/{self.name}/{url_extension}'
        response = requests.get(url, headers={
            'Authorization': f'Bearer {BEARER_TOKEN}'
        })
        if url_extension == 'environments' :
            return response.json().get('total_count')
        return len(response.json())
    
    def to_dict(self):
        return {
            'name': self.name,
            'stargazers_count': self.stargazers_count,
            'forks_count': self.forks_count,
            'commits_count': self.commits_count,
            'contributors_count': self.contributors_count,
            'branches_count': self.branches_count,
            'tags_count': self.tags_count,
            'releases_count': self.releases_count,
            'closed_issues_count': self.closed_issues_count,
            'environments_count' : self.enviroments_count
            
        }



In [116]:
print(repositories[0]['name'])

.allstar


In [ ]:
repo = Repository(repositories[1]['name'], repositories[1]['stargazers_count'], repositories[1]['forks_count'])
repo.to_dict()

{'name': 'docker-julia',
 'stargazers_count': 36,
 'forks_count': 12,
 'commits_count': 30,
 'contributors_count': 6,
 'branches_count': 1,
 'tags_count': 0,
 'releases_count': 0,
 'closed_issues_count': 0,
 'environments_count': 0}

In [118]:
repo_metrics = []
for repo in repositories:
    repo_metrics.append(Repository(repo['name'], repo['stargazers_count'], repo['forks_count']).to_dict())
repo_metrics


[{'name': '.allstar',
  'stargazers_count': 1,
  'forks_count': 0,
  'commits_count': 1,
  'contributors_count': 0,
  'branches_count': 1,
  'tags_count': 0,
  'releases_count': 0,
  'closed_issues_count': 0,
  'environments_count': 0},
 {'name': 'docker-julia',
  'stargazers_count': 36,
  'forks_count': 12,
  'commits_count': 30,
  'contributors_count': 6,
  'branches_count': 1,
  'tags_count': 0,
  'releases_count': 0,
  'closed_issues_count': 0,
  'environments_count': 0},
 {'name': 'docker-python',
  'stargazers_count': 2566,
  'forks_count': 980,
  'commits_count': 30,
  'contributors_count': 30,
  'branches_count': 30,
  'tags_count': 30,
  'releases_count': 30,
  'closed_issues_count': 0,
  'environments_count': 0},
 {'name': 'docker-rcran',
  'stargazers_count': 20,
  'forks_count': 14,
  'commits_count': 30,
  'contributors_count': 9,
  'branches_count': 8,
  'tags_count': 0,
  'releases_count': 0,
  'closed_issues_count': 0,
  'environments_count': 0},
 {'name': 'docker-rstat

In [125]:
class RepostioyStats:
    
    def __init__(self, repositories):
        self.repositories = repositories
        self.total_repos = len(repositories)

        self.total_stargazers_count = sum([repo.get('stargazers_count') for repo in repositories])
        self.median_stargazers_count = self.total_stargazers_count / self.total_repos
        
        self.total_forks_count = sum([repo.get('forks_count') for repo in repositories])
        self.median_forks_count = self.total_forks_count / self.total_repos
        
        self.total_commits_count = sum([repo.get('commits_count') for repo in repositories])
        self.median_commits_count = self.total_commits_count / self.total_repos

        self.total_contributors_count = sum([repo.get('contributors_count') for repo in repositories])
        self.median_contributors_count = self.total_contributors_count / self.total_repos

        self.total_branches_count = sum([repo.get('branches_count') for repo in repositories])
        self.median_branches_count = self.total_branches_count / self.total_repos

        self.total_tags_count = sum([repo.get('tags_count') for repo in repositories])
        self.median_tags_count = self.total_tags_count / self.total_repos

        self.total_releases_count = sum([repo.get('releases_count') for repo in repositories])
        self.median_releases_count = self.total_releases_count / self.total_repos

        self.total_closed_issues_count = sum([repo.get('closed_issues_count') for repo in repositories])
        self.median_closed_issues_count = self.total_closed_issues_count / self.total_repos
        
        self.total_enviroments_count = sum([repo.get('environments_count') for repo in repositories])
        self.median_enviroments_count = self.total_enviroments_count / self.total_repos
        
    
    def to_dict(self):
        return {
            # 'repositories' : self.repositories,
            'total_repos': self.total_repos,
            'total_stargazers_count': self.total_stargazers_count,
            'median_stargazers_count': self.median_stargazers_count,
            'total_forks_count': self.total_forks_count,
            'median_forks_count': self.median_forks_count,
            'total_commits_count': self.total_commits_count,
            'median_commits_count': self.median_commits_count,
            'total_contributors_count': self.total_contributors_count,
            'median_contributors_count': self.median_contributors_count,
            'total_branches_count': self.total_branches_count,
            'median_branches_count': self.median_branches_count,
            'total_tags_count': self.total_tags_count,
            'median_tags_count': self.median_tags_count,
            'total_releases_count': self.total_releases_count,
            'median_releases_count': self.median_releases_count,
            'total_closed_issues_count': self.total_closed_issues_count,
            'median_issues_count': self.median_closed_issues_count,
            'total_environments_count' : self.total_enviroments_count,
            'median_environments_count' : self.median_enviroments_count 
        }

In [130]:
repo_stats = RepostioyStats(repo_metrics)
repo_stats.to_dict()

{'total_repos': 12,
 'total_stargazers_count': 10469,
 'median_stargazers_count': 872.4166666666666,
 'total_forks_count': 2747,
 'median_forks_count': 228.91666666666666,
 'total_commits_count': 276,
 'median_commits_count': 23.0,
 'total_contributors_count': 177,
 'median_contributors_count': 14.75,
 'total_branches_count': 156,
 'median_branches_count': 13.0,
 'total_tags_count': 132,
 'median_tags_count': 11.0,
 'total_releases_count': 104,
 'median_releases_count': 8.666666666666666,
 'total_closed_issues_count': 0,
 'median_issues_count': 0.0,
 'total_environments_count': 0,
 'median_environments_count': 0.0}

In [132]:
# repo_stats.repositories